# Downloading dependencies

In [1]:
!pip install torch==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install torch-scatter==2.1.2 torch-sparse==0.6.18 torch-geometric==2.4.0 -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install ninja easydict pyyaml
!export CUDA_HOME=/usr/local/cuda-11.8/

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 649.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 2.1.0+cu118 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.1.0+cu118 which is incompatible.
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━

# Downloading ULTRA

In [2]:
!git clone https://github.com/DeepGraphLearning/ULTRA.git

Cloning into 'ULTRA'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 171 (delta 61), reused 58 (delta 58), pack-reused 92 (from 1)
Receiving objects: 100% (171/171), 7.62 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (85/85), done.


# Imports and Environment Setup

In [3]:
import sys
import os
import subprocess
sys.path.insert(0,'/content/ULTRA')
from ultra.datasets import InductiveDataset, IngramInductive

# ULTRA Datasets Building

In this section we are going to build our Custom Datasets. In particular, our datasets are built similarly to the ones used for testing Ingram. Thus, we are going to extend the class IngramInductive.

The data will be downloaded from the public git repository we previously built.

## F1 Dataset

In [4]:
code = """
class F1DatasetInductiveIngram(IngramInductive):
    urls = [
        "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-%s/inductive/train.txt",
        "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-%s/inductive/inference_graph.txt",
        "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-%s/inductive/inference_valid.txt",
        "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-%s/inductive/inference_test.txt",
    ]
    name = "f1_dataInductive"
"""

# We write add the new class to the datasets file of ULTRA in order to easily run the entire script
with open('/content/ULTRA/ultra/datasets.py', 'r') as f:
    content = f.read()

if "class F1DatasetInductiveIngram" not in content:
    with open('/content/ULTRA/ultra/datasets.py', 'a') as f:
        f.write(code)

# Zero-Shot ULTRA Testing

In this section we are going to test ULTRA on our custom datasets.

As we know, we can use any of the checkpoints in the ckpts directory given by ULTRA, thus, we are going to test ULTRA on several checkpoints.

## Testing ULTRA on F1 Dataset

### Using ULTRA trained on 3 graphs

In this section we are going to use the checkpoint: `ultra_3g.pth`, which has been trained on three graphs: `FB15k237`, `WN18RR`, `CoDExMedium` for 800,000 steps.

In [5]:
##### Here we setup the arguments used to run the script #####

script_path = "ULTRA/script/run.py" # we want to run ULTRA on a single dataset
config_path = "ULTRA/config/inductive/inference.yaml" # we want to test ULTRA on inference shot
dataset = "F1DatasetInductiveIngram" # we want to use the F1 dataset
version = "v2" # this is the version of input files
epochs = 0 # we want to perform zero-shot, thus we impose zero training epochs
bpe = "null" # batch-per-epochs
gpus = "[0]" # we run the experimet on a single GPU
checkpoint_path = "/content/ULTRA/ckpts/ultra_3g.pth" # we use ULTRA trained on 3 graphs

In [6]:
!python {script_path} -c {config_path} --dataset {dataset} --version {version} --epochs {epochs} --bpe {bpe} --gpus {gpus} --ckpt {checkpoint_path}

16:12:24   Random seed: 1024
16:12:24   Config file: ULTRA/config/inductive/inference.yaml
16:12:24   {'checkpoint': '/content/ULTRA/ckpts/ultra_3g.pth',
 'dataset': {'class': 'F1DatasetInductiveIngram',
             'root': '~/git/ULTRA/kg-datasets/',
             'version': 'v2'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_f

### Using ULTRA trained on 4 graphs

In this section we are going to use the checkpoint: `ultra_4g.pth`, which has been trained on three graphs: `FB15k237`, `WN18RR`, `CoDExMedium`, `NELL995` for 400,000 steps.

In [5]:
##### Here we setup the arguments used to run the script #####

script_path = "ULTRA/script/run.py" # we want to run ULTRA on a single dataset
config_path = "ULTRA/config/inductive/inference.yaml" # we want to test ULTRA on inference shot
dataset = "F1DatasetInductiveIngram" # we want to use the F1 dataset
version = "v2" # this is the version of input files
epochs = 0 # we want to perform zero-shot, thus we impose zero training epochs
bpe = "null" # batch-per-epochs
gpus = "[0]" # we run the experimet on a single GPU
checkpoint_path = "/content/ULTRA/ckpts/ultra_4g.pth" # we use ULTRA trained on 3 graphs

In [6]:
!python {script_path} -c {config_path} --dataset {dataset} --version {version} --epochs {epochs} --bpe {bpe} --gpus {gpus} --ckpt {checkpoint_path}

16:24:50   Random seed: 1024
16:24:50   Config file: ULTRA/config/inductive/inference.yaml
16:24:50   {'checkpoint': '/content/ULTRA/ckpts/ultra_4g.pth',
 'dataset': {'class': 'F1DatasetInductiveIngram',
             'root': '~/git/ULTRA/kg-datasets/',
             'version': 'v2'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_f

# Fine-Tuning ULTRA

## Testing ULTRA on F1 Dataset

### Using ULTRA trained on 3 graphs

In this section we are going to use the checkpoint: `ultra_3g.pth`, which has been trained on three graphs: `FB15k237`, `WN18RR`, `CoDExMedium` for 800,000 steps.

In [5]:
##### Here we setup the arguments used to run the script #####

script_path = "ULTRA/script/run.py" # we want to run ULTRA on a single dataset
config_path = "ULTRA/config/inductive/inference.yaml" # we want to test ULTRA on inference shot
dataset = "F1DatasetInductiveIngram" # we want to use the F1 dataset
version = "v2" # this is the version of input files
epochs = 5 # we want to perform pre-training, thus we impose a certain number of epochs
bpe = 100 # batch-per-epochs
gpus = "[0]" # we run the experimet on a single GPU
checkpoint_path = "/content/ULTRA/ckpts/ultra_3g.pth" # we use ULTRA trained on 3 graphs

In [6]:
!python {script_path} -c {config_path} --dataset {dataset} --version {version} --epochs {epochs} --bpe {bpe} --gpus {gpus} --ckpt {checkpoint_path}

16:30:36   Random seed: 1024
16:30:36   Config file: ULTRA/config/inductive/inference.yaml
16:30:36   {'checkpoint': '/content/ULTRA/ckpts/ultra_3g.pth',
 'dataset': {'class': 'F1DatasetInductiveIngram',
             'root': '~/git/ULTRA/kg-datasets/',
             'version': 'v2'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_f

### Using ULTRA trained on 4 graphs

In this section we are going to use the checkpoint: `ultra_4g.pth`, which has been trained on three graphs: `FB15k237`, `WN18RR`, `CoDExMedium`, `NELL995` for 400,000 steps.

In [5]:
##### Here we setup the arguments used to run the script #####

script_path = "ULTRA/script/run.py" # we want to run ULTRA on a single dataset
config_path = "ULTRA/config/inductive/inference.yaml" # we want to test ULTRA on inference shot
dataset = "F1DatasetInductiveIngram" # we want to use the F1 dataset
version = "v2" # this is the version of input files
epochs = 5 # we want to perform pre-training, thus we impose a certain number of epochs
bpe = 100 # batch-per-epochs
gpus = "[0]" # we run the experimet on a single GPU
checkpoint_path = "/content/ULTRA/ckpts/ultra_4g.pth" # we use ULTRA trained on 3 graphs

In [6]:
!python {script_path} -c {config_path} --dataset {dataset} --version {version} --epochs {epochs} --bpe {bpe} --gpus {gpus} --ckpt {checkpoint_path}

16:37:16   Random seed: 1024
16:37:16   Config file: ULTRA/config/inductive/inference.yaml
16:37:16   {'checkpoint': '/content/ULTRA/ckpts/ultra_4g.pth',
 'dataset': {'class': 'F1DatasetInductiveIngram',
             'root': '~/git/ULTRA/kg-datasets/',
             'version': 'v2'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_f